# Big Data L1 - Introduction to Apache Spark

# Чаплыгин Алексей 6131-010402D

# Задания Лабораторной работы в конце файла
[link to the end](#cell)

In [1]:
!pip install pyspark

# PySpark context initialization

In [2]:
from pyspark import SparkContext, SparkConf
from typing import NamedTuple
from datetime import datetime

In [3]:
app_name = "SimpleProject"
conf = SparkConf().setAppName(app_name).setMaster('local[1]')
sc = SparkContext(conf=conf)

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
sc

<SparkContext master=local[1] appName=SimpleProject>

#### Импорт данных и предобработка данных

In [5]:
!hadoop fs -ls /data

Found 11 items
-rwxr-xr-x   3 root root       2915 2022-01-20 05:48 /data/README.md
drwxr-xr-x   - root root         10 2022-01-20 06:20 /data/data
-rwxr-xr-x   3 root root        394 2022-01-20 05:48 /data/list_of_countries_sorted_gini.txt
-rwxr-xr-x   3 root root   19459967 2022-01-20 05:48 /data/nycTaxiFares.gz
-rwxr-xr-x   3 root root   84135506 2022-01-20 05:48 /data/nycTaxiRides.gz
-rwxr-xr-x   3 root root   79500408 2022-01-20 05:48 /data/nyctaxi.csv
-rwxr-xr-x   3 root root   74162295 2022-01-20 05:49 /data/posts_sample.xml
-rwxr-xr-x   3 root root      40269 2022-01-20 05:49 /data/programming-languages.csv
-rwxr-xr-x   3 root root       5647 2022-01-20 05:49 /data/stations.csv
-rwxr-xr-x   3 root root   80208831 2022-01-20 05:49 /data/trips.csv
-rwxr-xr-x   3 root root    5315699 2022-01-20 05:49 /data/warandsociety.txt


In [6]:
trip_source_data = sc.textFile("/mnt/data/trips.csv")
station_source_data = sc.textFile("/mnt/data/stations.csv")

In [7]:
trip_source_data.count()

669960

In [8]:
station_source_data.count()

71

In [9]:
!head /mnt/data/trips.csv

id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
4576,63,8/29/2013 13:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
4607,13,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103
4927,103,8/29/2013 18:54,Golden Gate at Polk,59,8/29/2013 18:56,Golden Gate at Polk,59,527,Subscriber,94109
4500,109,8/29/2013 13:25,Santa Clara at Almaden,4,8/29/2013 13:27,Adobe on Almaden,5,679,Subscriber,95112
4563,111,8/29/2013 14:02,San Salvador at 1st,8,8/29/2013 14:04,San Salvador at 1st,8,687,Subscriber,9511

In [10]:
!head /mnt/data/stations.csv

id,name,lat,long,dock_count,city,installation_date
2,San Jose Diridon Caltrain Station,37.329732,-121.90178200000001,27,San Jose,8/6/2013
3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,8/5/2013
6,San Pedro Square,37.336721000000004,-121.894074,15,San Jose,8/7/2013
7,Paseo de San Antonio,37.333798,-121.88694299999999,15,San Jose,8/7/2013
8,San Salvador at 1st,37.330165,-121.88583100000001,15,San Jose,8/5/2013
9,Japantown,37.348742,-121.89471499999999,15,San Jose,8/5/2013
10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013


In [11]:
# определяем заголовки из первой строки в данных
trip_headers = trip_source_data.first()
station_headers = station_source_data.first()
# строки заголовков делим на столбцы по ","
trip_header_list = list(enumerate(trip_headers.split(",")))
station_header_list = list(enumerate(station_headers.split(",")))

In [12]:
print(trip_headers)
print(station_headers)
print(trip_header_list)
print(station_header_list)

id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
id,name,lat,long,dock_count,city,installation_date
[(0, 'id'), (1, 'duration'), (2, 'start_date'), (3, 'start_station_name'), (4, 'start_station_id'), (5, 'end_date'), (6, 'end_station_name'), (7, 'end_station_id'), (8, 'bike_id'), (9, 'subscription_type'), (10, 'zip_code')]
[(0, 'id'), (1, 'name'), (2, 'lat'), (3, 'long'), (4, 'dock_count'), (5, 'city'), (6, 'installation_date')]


In [13]:
# убираем заголовки от строк с данными
trips = trip_source_data.filter(lambda row: row != trip_headers)
stations = station_source_data.filter(lambda row: row != station_headers)

# строки с данными делим на столбцы по ","
trips_data = trips.map(lambda x: x.split(','))
stations_data = stations.map(lambda x: x.split(','))

In [14]:
trips_data.take(5)

[['4576',
  '63',
  '8/29/2013 13:13',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '13',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138'],
 ['4130',
  '71',
  '8/29/2013 10:16',
  'Mountain View City Hall',
  '27',
  '8/29/2013 10:17',
  'Mountain View City Hall',
  '27',
  '48',
  'Subscriber',
  '97214'],
 ['4251',
  '77',
  '8/29/2013 11:29',
  'San Jose City Hall',
  '10',
  '8/29/2013 11:30',
  'San Jose City Hall',
  '10',
  '26',
  'Subscriber',
  '95060'],
 ['4299',
  '83',
  '8/29/2013 12:02',
  'South Van Ness at Market',
  '66',
  '8/29/2013 12:04',
  'Market at 10th',
  '67',
  '319',
  'Subscriber',
  '94103']]

In [15]:
stations_data.take(5)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013'],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '8/5/2013'],
 ['4',
  'Santa Clara at Almaden',
  '37.333988',
  '-121.894902',
  '11',
  'San Jose',
  '8/6/2013'],
 ['5',
  'Adobe on Almaden',
  '37.331415',
  '-121.8932',
  '19',
  'San Jose',
  '8/5/2013'],
 ['6',
  'San Pedro Square',
  '37.336721000000004',
  '-121.894074',
  '15',
  'San Jose',
  '8/7/2013']]

In [16]:
# Инициализация модели Trips
def init_trip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str
        
    for trip in trips:
        try:
            yield Trip(                             
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            ) 
        except:
            pass

# Инициализация модели Trips Stations
def init_station(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    
    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

In [17]:
# Формируем объекты
trips_objects = trips_data.mapPartitions(init_trip)
station_objects = stations_data.mapPartitions(init_station)

In [18]:
trips_objects.first()

Trip(trip_id=4576, duration=63, start_date=datetime.datetime(2013, 8, 29, 13, 13), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 14, 14), end_station_name='South Van Ness at Market', end_station_id='66', bike_id=520, subscription_type='Subscriber', zip_code='94127')

In [19]:
station_objects.first()

Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0))

# Задание
<a id="cell"></a>

## 1. Найти велосипед с максимальным пробегом.

In [20]:
trips_by_bike = trips_objects.keyBy(lambda trip: trip.bike_id)

In [21]:
query = trips_by_bike\
    .mapValues(lambda trip: trip.duration)\
    .reduceByKey(lambda trip1, trip2: trip1 + trip2)

In [22]:
query.map(lambda x: x[::-1]).top(5)

[(18611693, 535),
 (3933272, 466),
 (2409014, 613),
 (2253019, 526),
 (2248886, 415)]

In [23]:
top_1_bike_id = query.map(lambda x: x[::-1]).top(1)[0][1]
top_1_bike_id

535

## 2. Найти наибольшее расстояние между станциями.


In [24]:
trips_stations = trips_objects\
    .filter(lambda trip: trip.start_station_name != trip.end_station_name)\
    .keyBy(lambda trip: (trip.start_station_name, trip.end_station_name))\
    .mapValues(lambda trip: trip.duration)

In [25]:
query = trips_stations\
    .aggregateByKey(
        (0.0, 0.0),
        lambda acc, value: (acc[0] + value, acc[1] + 1),
        lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]),
    )\
    .mapValues(lambda values: values[0] / values[1])

In [26]:
query.map(lambda x: x[::-1]).top(5)

[(229914.0, ('Redwood City Medical Center', 'SJSU - San Salvador at 9th')),
 (179212.5, ('Castro Street and El Camino Real', 'Howard at 2nd')),
 (169308.0,
  ('Santa Clara County Civic Center', 'California Ave Caltrain Station')),
 (156461.03603603604, ('South Van Ness at Market', '2nd at Folsom')),
 (101207.5,
  ('Mountain View Caltrain Station', 'San Jose Diridon Caltrain Station'))]

## 3. Найти путь велосипеда с максимальным пробегом через станции.



In [27]:
query = trips_objects.filter(lambda trip: trip.bike_id == top_1_bike_id)\
    .sortBy(lambda trip: trip.start_date)\
    .map(lambda trip: (trip.start_station_name, trip.end_station_name))    

In [28]:
query.collect()

[('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)'),
 ('San Francisco Caltrain (Townsend at 4th)',
  'San Francisco Caltrain 2 (330 Townsend)'),
 ('San Francisco Caltrain 2 (330 Townsend)', 'Market at Sansome'),
 ('Market at Sansome', '2nd at South Park'),
 ('2nd at Townsend', 'Davis at Jackson'),
 ('San Francisco City Hall', 'Civic Center BART (7th at Market)'),
 ('Civic Center BART (7th at Market)', 'Post at Kearney'),
 ('Post at Kearney', 'Embarcadero at Sansome'),
 ('Embarcadero at Sansome', 'Washington at Kearney'),
 ('Washington at Kearney', 'Market at Sansome'),
 ('Market at Sansome', 'Market at Sansome'),
 ('Market at Sansome', '2nd at Folsom'),
 ('2nd at Folsom', '2nd at Townsend'),
 ('Temporary Transbay Terminal (Howard at Beale)', '2nd at Townsend'),
 ('2nd at Townsend', 'Embarcadero at Sansome'),
 ('Embarcadero at Sansome', 'Clay at Battery'),
 ('Clay at Battery', 'Harry Bridges Plaza (Ferry Building)'),
 ('Harry Bridges Plaza (Ferry Building)', 'Clay at Batter

## 4. Найти количество велосипедов в системе.

In [29]:
query = trips_objects.map(lambda trip: trip.bike_id)

In [30]:
query.distinct().count()

700

## 5. Найти пользователей потративших на поездки более 3 часов.

_Т.к. среди наших данных нет информации о пользователях, пусть оно звучит следующим образом:_

__Найти номера поездок, которые длились более 3 часов.__

In [31]:
bound = 3 * 60 * 60

In [32]:
query = trips_objects\
    .filter(lambda trip: trip.duration > bound)\
    .map(lambda trip: trip.trip_id)

In [33]:
query.take(5)

[4639, 4637, 4528, 4363, 4193]